# Data Engineer Certification - Practical Exam - Supplement Experiments

FitX makes personalized supplements tailored to individual health needs.

FitX aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at FitX plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps FitX provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

FitX currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [175]:
# Use as many python cells as you wish to write your code
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("HealthDataAnalysis").getOrCreate()

In [176]:
import pandas as pd

# Defs

In [177]:
def create_sql_views(csv_files):
    temp_views = [f.split(".")[0] for f in csv_files]
    
    # Import CSV files and create temporary views
    for file, view in zip(csv_files, temp_views):
        df = spark.read.csv(file, header=True, inferSchema=True)
        df.createOrReplaceTempView(view)
        

def merge_all_data(user_health_data, supplement_usage, experiments, user_profiles):
    csv_files = [user_health_data, supplement_usage, experiments, user_profiles] 
    create_sql_views(csv_files)
    
    data_merge_sdf = spark.sql('''

        select 
            -- 1
            up.user_id

            -- 2
            , cast(uhd.date as date) as date

            -- 3
            , up.email

            -- 4
            , case 
                when age is null then 'Unknown'
                when age < 18 then 'Under 18'
                when age between 18 and 25 then '18-25'
                when age between 26 and 35 then '26-35'
                when age between 36 and 45 then '36-45'
                when age between 46 and 55 then '46-55'
                when age between 56 and 65 then '56-65'
                else 'Over 65'
            end as user_age_group

            -- 5
            , e.name as experiment_name

            -- 6
            , coalesce(su.supplement_name, 'No intake') as supplement_name

            -- 7
            , coalesce(su.dosage / 1000, null) as dosage_grams

            -- 8
            , su.is_placebo

            -- 9
            , uhd.average_heart_rate

            -- 10
            , uhd.average_glucose

            -- 11
            , lower(uhd.sleep_hours) as sleep_hours

            -- 12
            , uhd.activity_level

        from user_profiles up
            left join user_health_data uhd 
                on up.user_id = uhd.user_id
            left join supplement_usage su
                on up.user_id = su.user_id 
                    and uhd.date = su.date
            left join experiments e 
                on su.experiment_id = e.experiment_id 

        where 
            up.user_id is not null 
            and uhd.date is not null 
            and up.email is not null
    ''')
    
    data_merge_sdf.createOrReplaceTempView('data_merge')

    df = data_merge_sdf.toPandas()
    
    df['date'] = pd.to_datetime(df['date'])
    df['user_id'] = df['user_id'].astype(str)
    df['email'] = df['email'].astype(str)
    df['user_age_group'] = df['user_age_group'].astype(str)
    df['experiment_name'] = df['experiment_name'].astype(str)
    df['supplement_name'] = df['supplement_name'].replace('null', None).astype(str)
    df['dosage_grams'] = pd.to_numeric(df['dosage_grams'])
    df['is_placebo'] = df['is_placebo'].replace('null', None).astype('boolean')
    df['average_heart_rate'] = pd.to_numeric(df['average_heart_rate'])
    df['average_glucose'] = pd.to_numeric(df['average_glucose'])
    df['sleep_hours'] = df['sleep_hours'].astype(str)
    df['activity_level'] = pd.to_numeric(df['activity_level'])
    
    return df


# Look at Data

In [178]:
# user_health_data
spark.sql('''
    select *
    from user_health_data
    limit 10
''').toPandas()

user_id       date  ...  sleep_hours  activity_level
0  c6ae338a-9f95-481c-a88d-24a58bc8fc71 2018-01-31  ...         8.8h               1
1  c6ae338a-9f95-481c-a88d-24a58bc8fc71 2018-02-28  ...         8.0H               3
2  c6ae338a-9f95-481c-a88d-24a58bc8fc71 2018-03-31  ...        11.9h               1
3  c6ae338a-9f95-481c-a88d-24a58bc8fc71 2018-04-30  ...         5.1h               1
4  5346f1dc-30f7-4e3a-9d35-eec6cb8835fa 2018-01-31  ...         9.6h               2
5  5346f1dc-30f7-4e3a-9d35-eec6cb8835fa 2018-02-28  ...         7.2H               2
6  5346f1dc-30f7-4e3a-9d35-eec6cb8835fa 2018-03-31  ...         7.4h               2
7  5346f1dc-30f7-4e3a-9d35-eec6cb8835fa 2018-04-30  ...         9.7h               4
8  5541289d-fa9f-4aef-9504-1e11d940efc5 2018-01-31  ...         9.5H               4
9  5541289d-fa9f-4aef-9504-1e11d940efc5 2018-02-28  ...        11.9h               3

[10 rows x 6 columns]

In [179]:
# supplement_usage
spark.sql('''
    select distinct dosage_unit
    from supplement_usage
    limit 10
''').toPandas()

dosage_unit
0          mg

In [180]:
# experiments
spark.sql('''
    select *
    from experiments
    limit 10
''').toPandas()

experiment_id  ...                                        description
0  d655666f-725d-4312-96b4-df25045dd08e  ...  Improve how long aerobic physical activities c...
1  33c1bd33-b41a-4773-a001-873307be57e9  ...  Improve the maximum strength in different ranges.
2  85d2741f-7964-49d4-bc1d-62f0a50d8a59  ...  Examine the impact of different conditions on ...
3  14fb6430-c228-407c-ba66-a35588b98276  ...  Affect the ability to concentrate and maintain...
4  2a8e0839-ed8f-4409-8aab-c4abf7cb1185  ...  Evaluate techniques to improve short-term and ...
5  ce7f2178-4f42-4511-ab41-06b8cf717f7c  ...  Investigate recovery speeds after exertion, fo...

[6 rows x 3 columns]

In [181]:
# user_profiles
spark.sql('''
    select *
    from user_profiles
    limit 10
''').toPandas()

user_id                  email   age
0  c6ae338a-9f95-481c-a88d-24a58bc8fc71       hi_1@example.com  68.0
1  5346f1dc-30f7-4e3a-9d35-eec6cb8835fa       hi_2@myemail.com  64.0
2  5541289d-fa9f-4aef-9504-1e11d940efc5    hello_3@example.com  40.0
3  f8461502-54b8-4388-ae0d-9095cfc0426f        hi4@myemail.com  33.0
4  9e4781d1-111b-4318-907f-63c58da6cd89     user_5@myemail.com  48.0
5  c7612504-ff78-4b95-aa89-a7f28b7a77b2       hi_6@myemail.com  28.0
6  4e2986dc-4f81-484e-8865-03513b668365   contact7@example.com  33.0
7  8058e377-a41f-4138-afd5-3b39bbf750e2  contact_8@myemail.com   NaN
8  28c35ac0-7140-4a88-8e8c-fdd807bc1259     hello9@example.com  21.0
9  7a9e935e-1f5f-4429-8e20-1e1a001a7986      user_10@email.com  57.0

# proto queries 

In [182]:
%%time 

spark.sql('''

    select count(*) - count(distinct *)
    from raw_inner_join

''').toPandas()

CPU times: user 6.49 ms, sys: 160 µs, total: 6.65 ms
Wall time: 374 ms


(count(1) - count(DISTINCT user_id, date, email, user_age_group, experiment_name, supplement_name, dosage_grams, is_placebo, average_heart_rate, average_glucose, sleep_hours, activity_level))
0                                                721

In [183]:
%%time 

spark.sql('''

    select *
    
    from raw_inner_join

''').toPandas()

CPU times: user 20.5 ms, sys: 11.7 ms, total: 32.2 ms
Wall time: 234 ms


user_id  ... activity_level
0     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
1     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              3
2     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
3     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
4     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
...                                    ...  ...            ...
2716  9c083dc3-b789-4cee-87fb-9887ff4b4e29  ...              3
2717  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              1
2718  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4
2719  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              3
2720  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4

[2721 rows x 12 columns]

In [184]:
# Check for duplicate rows
duplicates = spark.sql('''
    select 
        user_id, date, email, user_age_group, experiment_name, supplement_name, dosage_grams, is_placebo, average_heart_rate, average_glucose, sleep_hours, activity_level,
        count(*) as cnt
    from data_merge
    group by 
        user_id, date, email, user_age_group, experiment_name, supplement_name, dosage_grams, is_placebo, average_heart_rate, average_glucose, sleep_hours, activity_level
    having cnt > 1
''')

duplicates.toPandas()

Empty DataFrame
Columns: [user_id, date, email, user_age_group, experiment_name, supplement_name, dosage_grams, is_placebo, average_heart_rate, average_glucose, sleep_hours, activity_level, cnt]
Index: []

In [185]:
%%time 

spark.sql('''

    select count(*) as duplicate_count
    from (
        select 
            count(*) as cnt
        from raw_inner_join
        group by 
            user_id, date, email, user_age_group, experiment_name, supplement_name, dosage_grams, is_placebo, average_heart_rate, average_glucose, sleep_hours, activity_level
        having cnt > 1
    )

''').toPandas()

CPU times: user 7.57 ms, sys: 232 µs, total: 7.8 ms
Wall time: 307 ms


duplicate_count
0                0

In [186]:
%%time 

df = spark.sql('''

    select *
    
    from raw_inner_join

''').toPandas()

print('len = ', len(df))
df

len =  2721
CPU times: user 32.7 ms, sys: 0 ns, total: 32.7 ms
Wall time: 332 ms


user_id  ... activity_level
0     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
1     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              3
2     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
3     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
4     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
...                                    ...  ...            ...
2716  9c083dc3-b789-4cee-87fb-9887ff4b4e29  ...              3
2717  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              1
2718  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4
2719  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              3
2720  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4

[2721 rows x 12 columns]

In [187]:
%%time 

spark.sql('''

    select *
    
    from raw_inner_join

''').toPandas()

CPU times: user 29.1 ms, sys: 7.93 ms, total: 37.1 ms
Wall time: 239 ms


user_id  ... activity_level
0     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
1     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              3
2     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
3     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
4     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
...                                    ...  ...            ...
2716  9c083dc3-b789-4cee-87fb-9887ff4b4e29  ...              3
2717  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              1
2718  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4
2719  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              3
2720  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4

[2721 rows x 12 columns]

# Merge All Data

In [188]:
%%time

merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')

CPU times: user 571 ms, sys: 0 ns, total: 571 ms
Wall time: 1.43 s


user_id  ... activity_level
0     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
1     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              3
2     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
3     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
4     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
...                                    ...  ...            ...
2716  9c083dc3-b789-4cee-87fb-9887ff4b4e29  ...              3
2717  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              1
2718  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4
2719  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              3
2720  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4

[2721 rows x 12 columns]